In [1]:
import pandas as pd; pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True) # sns.set(style='darkgrid')

import geopandas as gpd # http://geopandas.org/gallery/plotting_basemap_background.html#sphx-glr-gallery-plotting-basemap-background-py
import contextily as ctx

import shapely.wkt # https://stackoverflow.com/questions/51855917/shapely-polygon-from-string
from shapely.geometry import Point, LineString, MultiLineString, Polygon
from shapely.ops import cascaded_union

In [2]:
df = pd.read_csv('NYPD_Arrests_Data__Historic_.csv')

In [3]:
df.shape

(4798339, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4798339 entries, 0 to 4798338
Data columns (total 18 columns):
ARREST_KEY           int64
ARREST_DATE          object
PD_CD                float64
PD_DESC              object
KY_CD                float64
OFNS_DESC            object
LAW_CODE             object
LAW_CAT_CD           object
ARREST_BORO          object
ARREST_PRECINCT      int64
JURISDICTION_CODE    float64
AGE_GROUP            object
PERP_SEX             object
PERP_RACE            object
X_COORD_CD           float64
Y_COORD_CD           float64
Latitude             float64
Longitude            float64
dtypes: float64(7), int64(2), object(9)
memory usage: 659.0+ MB


In [ ]:
def Data_Stat(df, n_row_view = 3):
    types = pd.Series({c:str(df[c].dtype) for c in df.columns}) # variable type
    total = df.isnull().sum() # the number of null values
    percent = 100 * total / df.isnull().count() # the percentage of null values
    n_unique = df.nunique() # the number of unique values
    
    tt = np.transpose(pd.concat([types, total, percent, n_unique], axis=1, keys=['Types', '# of nulls', '% of nulls', '# of uniques']))
    return pd.concat([df.head(n_row_view), tt], axis = 0)

Data_Stat(df)
# df.head()

In [ ]:
df['ARREST_DATE'] = pd.to_datetime(df['ARREST_DATE'], format='%m/%d/%Y')
df['year'] = df['ARREST_DATE'].dt.year
df['month'] = df['ARREST_DATE'].dt.month
df['day'] = df['ARREST_DATE'].dt.day
df['dayofweek'] = df['ARREST_DATE'].dt.dayofweek
df['hour'] = df['ARREST_DATE'].dt.hour

df['location'] = df[['Longitude','Latitude']].apply(lambda x: Point(x[0],x[1]), axis=1)

df['count'] = 1

df.head()

In [ ]:
df.set_index('ARREST_DATE').resample('M').count().plot(y='count')

In [ ]:
df.set_index('ARREST_DATE').resample('D').count().plot(y='count')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df.set_index('ARREST_DATE').resample('M')['count'].count(), 
                            model='additive') #ref) https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/
result.plot()
plt.show()

In [ ]:
# https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz
df_precinct = gpd.read_file('Police Precincts/geo_export_bf3a580e-5e32-463e-a39e-5c7a9c0405cf.shp')
print(df_precinct.shape)
df_precinct.head(3)

In [ ]:
for year in [2006,2012,2018]:
    fig, ax = plt.subplots(1, figsize=(10, 6))
    df_precinct.join(
        df.loc[df['year']==year].groupby('ARREST_PRECINCT')['count'].count(),
        on='precinct', how='left'
    ).plot(ax=ax, column='count', cmap='OrRd', linewidth=0.2, edgecolor='0.2', legend=True, figsize=(10,6))
    ax.axis('off')
    plt.title('year = %s'%year)
    plt.show()

In [ ]:
# data source) https://www1.nyc.gov/site/planning/planning-level/nyc-population/current-future-populations.page
nyc_pop = pd.DataFrame({'year':       [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], 
                        'population': [8.190,8.273,8.348,8.399,8.437,8.468,8.476,8.438,8.399]})
nyc_pop['population'] = nyc_pop['population'] * 1000000

In [ ]:
nyc_pop.plot(x='year', y='population', marker='o')

In [ ]:
arrest_count = df.groupby('year')['count'].sum()
arrest_count

In [ ]:
nyc_arrests = nyc_pop.join(arrest_count, on='year', how='left')
nyc_arrests['arrest_rate'] = nyc_arrests['count'] / nyc_arrests['population'] * 100000 # https://oag.ca.gov/sites/all/files/agweb/pdfs/cjsc/stats/computational_formulas.pdf
nyc_arrests.plot(x='year', y='arrest_rate', marker='o'); plt.show()
nyc_arrests

In [ ]:
df_month = df.groupby(['year','month']).count().reset_index(drop=False)
df_month = df_month.join(nyc_pop.set_index('year'), on='year', how='left')
df_month['arrest_rate'] = df_month['count'] / df_month['population'] * 100000
df_month_2015_2018 = df_month.loc[df_month['year']>=2015]

sns.lmplot(x='year', y='arrest_rate', data=df_month_2015_2018, ci=95, x_jitter=0.1) # https://seaborn.pydata.org/tutorial/regression.html
plt.xticks([2015,2016,2017,2018])
plt.show()

In [ ]:
df_day = df.groupby(['year','month','day']).count().reset_index(drop=False)
df_day = df_day.join(nyc_pop.set_index('year'), on='year', how='left')
df_day['arrest_rate'] = df_day['count'] / df_day['population'] * 100000
df_day_2015_2018 = df_day.loc[df_day['year']>=2015]

sns.lmplot(x='year', y='arrest_rate', data=df_day_2015_2018, ci=95, x_jitter=0.1) # https://seaborn.pydata.org/tutorial/regression.html
plt.xticks([2015,2016,2017,2018])
plt.show()

In [ ]:
PD_DESC_freq = df.loc[df['year']==2018,'PD_DESC'].value_counts()
PD_DESC_freq.reset_index().rename(columns={'index':'PD_DESC','PD_DESC':'arrest_count_2018'}).head(11)

In [ ]:
PD_DESC_freq_top5 = PD_DESC_freq.index[0:5]
PD_DESC_freq_top5

In [ ]:
df.loc[df['PD_DESC'].apply(lambda x: x in PD_DESC_freq_top5),['PD_DESC','LAW_CAT_CD']]\
  .drop_duplicates().sort_values('PD_DESC').reset_index(drop=True)

In [ ]:
data = df.loc[df['PD_DESC'].apply(lambda x: x in PD_DESC_freq_top5)].\
          groupby(['year','PD_DESC'])['count'].count().\
          reset_index()

fig, ax = plt.subplots(figsize=(15,7))
sns.pointplot(ax=ax, x='year', y='count', hue='PD_DESC', data=data, capsize=.2, dodge=True)
plt.show()

In [ ]:
data = df.loc[df['ARREST_PRECINCT'].apply(lambda x: x in [19,73])].\
          groupby(['year','ARREST_PRECINCT'])['count'].count().\
          reset_index()

fig, ax = plt.subplots(figsize=(15,7))
sns.pointplot(ax=ax, x='year', y='count', hue='ARREST_PRECINCT', data=data, capsize=.2, dodge=True)
plt.show()

In [ ]:
df['ratio_felony'] = df['LAW_CAT_CD'].apply(lambda x: x=='F' if pd.notnull(x) else False)
data2 = df.loc[df['ARREST_PRECINCT'].apply(lambda x: x in [19,73])]

fig, ax = plt.subplots(figsize=(15,7))
sns.pointplot(ax=ax, x='year', y='ratio_felony', hue='ARREST_PRECINCT', data=data2, ci=95, capsize=.2, dodge=True)
plt.show()